**Commands to install required packages**
- pip install pandas mlxtend numpy

**Utility functions**

In [23]:
def do_while(check_condition, action):
  action()
  while check_condition():
    action()

# Data Selection

In [24]:
### Listing all datasets stored in the ./datasets folder

from os import listdir
from os.path import isfile, join

path = './datasets/'

datasets = [f.split('.')[0] for f in listdir(path) if isfile(join(path, f)) and f.split('.')[1] == 'csv']
datasets = dict(zip(range(1, len(datasets) + 1), datasets))
print(datasets)

{1: 'Best Buy', 2: 'Amazon', 3: 'Generic', 4: 'Nike', 5: 'K-Mart', 6: 'Custom'}


In [25]:
dataset = 0
attempted = 0

def read_dataset_input():
  """
      - On the first attempt, prompts the user to select a dataset.
      - On subsequent attempts, informs the user of an invalid selection if needed.
      - If the user's input matches a key in the `datasets` dictionary, it prints the selected dataset.
      
      Returns:
      None
  """
  global attempted, dataset
  print("Invalid selection. Try again. \n") if attempted > 0 else print("Select a dataset: \n")
  for k,v in datasets.items():
    print(k, ': ', v)
  attempted += 1
  dataset = int(input())
  if dataset in datasets.keys(): print("You selected: ", datasets[dataset])

input_read_condition = lambda: dataset not in datasets.keys()
do_while(input_read_condition, read_dataset_input)


Select a dataset: 

1 :  Best Buy
2 :  Amazon
3 :  Generic
4 :  Nike
5 :  K-Mart
6 :  Custom
You selected:  Best Buy


# Defining support and confidence

In [26]:
support = None
confidence = None

def read_threshold():
  """
    - Prompts the user to enter a value for the support threshold.
    - Prompts the user to enter a value for the confidence threshold.

    Returns:
    None
  """
  global support, confidence
  support = input("Enter support threshold between 0.1 to 100 : ")
  confidence = input("Enter confidence threshold between 0.1 to 100 : ")

def threshold_read_condition():
  """
    - Attempts to convert `support` and `confidence` to floats.
    - If conversion fails (i.e., invalid input types), prints an error message and returns True to
      indicate the need for retrying.
    - Checks whether the values for `support` and `confidence` fall within the valid range [0, 1].
    - Returns False if both values are valid, otherwise returns True and prints an error message.

    Returns:
    bool: True if input is invalid (either due to type or out-of-range values), False otherwise.
  """
  global support, confidence
  try:
    support = float(support)
    confidence = float(confidence)
  except ValueError:
    print("Invalid input type. Try again.")
    return True
  if 0 < support <= 1 and 0 < confidence <= 1:
    return False
  print("Invalid input range. Try again.")
  return True
do_while(threshold_read_condition, read_threshold)

print("Generating association rules for ", datasets[dataset], " dataset with support: ", support, " and confidence: ", confidence)

Generating association rules for  Best Buy  dataset with support:  0.5  and confidence:  0.5


# Reading Transactions

In [27]:
import pandas as pd

raw_dataset = pd.read_csv('./datasets/' + datasets[dataset] + '.csv', usecols=[1])
transactions_list = [transaction[0].split(',') for transaction in raw_dataset.values.tolist()]
transactions_list = [[item.strip() for item in transaction] for transaction in transactions_list]

print(raw_dataset)

                                          Transaction
0   Desk Top, Printer, Flash Drive, Microsoft Offi...
1   Lab Top, Flash Drive, Microsoft Office, Lab To...
2   Lab Top, Printer, Flash Drive, Microsoft Offic...
3   Lab Top, Printer, Flash Drive, Anti-Virus, Ext...
4     Lab Top, Flash Drive, Lab Top Case, Anti-Virus 
5    Lab Top, Printer, Flash Drive, Microsoft Office 
6   Desk Top, Printer, Flash Drive, Microsoft Office 
7           Lab Top, External Hard-Drive, Anti-Virus 
8   Desk Top, Printer, Flash Drive, Microsoft Offi...
9   Digital Camera , Lab Top, Desk Top, Printer, F...
10  Lab Top, Desk Top, Lab Top Case, External Hard...
11  Digital Camera , Lab Top, Lab Top Case, Extern...
12                         Digital Camera , Speakers 
13  Digital Camera , Desk Top, Printer, Flash Driv...
14  Printer, Flash Drive, Microsoft Office, Anti-V...
15  Digital Camera, Flash Drive, Microsoft Office,...
16            Digital Camera , Lab Top, Lab Top Case 
17           Digital Camera 

# Arpiori Using mlxtend

In [28]:
import time

In [29]:
from mlxtend.frequent_patterns import apriori, association_rules

start_time = time.time()

all_items = set(item for sublist in transactions_list for item in sublist)

df = pd.DataFrame([{item: (item in transaction) for item in all_items} for transaction in transactions_list])

frequent_itemsets = apriori(df, min_support=support, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)

rules_sorted = rules.sort_values(by='lift', ascending=False)

rules_filtered = rules[['antecedents', 'consequents', 'support', 'confidence']]
rules_filtered.columns = ['Antecedents', 'Consequents', 'Support', 'Confidence']

print(rules_filtered)

end_time = time.time()
print("Time taken: ", end_time - start_time, " seconds")

           Antecedents         Consequents  Support  Confidence
0   (Microsoft Office)       (Flash Drive)     0.55    1.000000
1        (Flash Drive)  (Microsoft Office)     0.55    0.846154
2        (Flash Drive)           (Printer)     0.50    0.769231
3            (Printer)       (Flash Drive)     0.50    1.000000
4         (Anti-Virus)       (Flash Drive)     0.50    0.714286
5        (Flash Drive)        (Anti-Virus)     0.50    0.769231
6            (Lab Top)      (Lab Top Case)     0.50    0.833333
7       (Lab Top Case)           (Lab Top)     0.50    0.714286
8            (Lab Top)        (Anti-Virus)     0.50    0.833333
9         (Anti-Virus)           (Lab Top)     0.50    0.714286
10      (Lab Top Case)        (Anti-Virus)     0.60    0.857143
11        (Anti-Virus)      (Lab Top Case)     0.60    0.857143
Time taken:  0.008301973342895508  seconds


# FP-growth Using mlxtend

In [30]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

start_time = time.time()
frequent_itemsets = fpgrowth(df, min_support=support, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)

rules_filtered = rules[['antecedents', 'consequents', 'support', 'confidence']]
rules_filtered.columns = ['Antecedents', 'Consequents', 'Support', 'Confidence']

print(rules_filtered)

end_time = time.time()
print("Time taken: ", end_time - start_time, " seconds")

           Antecedents         Consequents  Support  Confidence
0         (Anti-Virus)       (Flash Drive)     0.50    0.714286
1        (Flash Drive)        (Anti-Virus)     0.50    0.769231
2   (Microsoft Office)       (Flash Drive)     0.55    1.000000
3        (Flash Drive)  (Microsoft Office)     0.55    0.846154
4        (Flash Drive)           (Printer)     0.50    0.769231
5            (Printer)       (Flash Drive)     0.50    1.000000
6       (Lab Top Case)        (Anti-Virus)     0.60    0.857143
7         (Anti-Virus)      (Lab Top Case)     0.60    0.857143
8            (Lab Top)      (Lab Top Case)     0.50    0.833333
9       (Lab Top Case)           (Lab Top)     0.50    0.714286
10           (Lab Top)        (Anti-Virus)     0.50    0.833333
11        (Anti-Virus)           (Lab Top)     0.50    0.714286
Time taken:  0.00527191162109375  seconds


# Implementing Apiori

**Generating frequent items**

In [31]:
from itertools import combinations

min_sup = support * len(transactions_list)
freq_itemset_support = {}


def count_item_freq(itemsets):
  """
    - Iterates over each transaction in `transactions_list` and checks whether each itemset
      is present in the transaction.
    - If all items in an itemset are found within a transaction, increments the count for that
      itemset in the `itemset_support` dictionary.
    - Uses the `itemset_support` dictionary to store the frequency of each itemset.

    Returns:
    dict: A dictionary where the keys are itemsets and the values are their corresponding frequencies.
  """
  itemset_support = {}
  for transaction in transactions_list:
    for itemset in itemsets:
      for item in itemset:
        if item not in transaction:
          break
      else:
        itemset_support[itemset] = itemset_support.get(itemset, 0) + 1
  return itemset_support


def prune_items(last_freq_itemset):
  """
    - Iterates over each itemset and its support in `last_freq_itemset`.
    - Filters out itemsets whose support is below `min_sup`.
    - For each retained itemset, calculates its relative support as the ratio of its count to the
      total number of transactions.

    Returns:
    dict: A dictionary where the keys are itemsets and the values are their relative support
          (calculated as support count divided by the total number of transactions), for itemsets
          that meet or exceed the minimum support threshold.
  """
  return {itemset:(sup/len(transactions_list)) for itemset,sup in last_freq_itemset.items() if sup >= min_sup}

def make_n_itemset(n_itemset):
  """
    - Extracts unique items from the provided `n_itemset`.
      - Creates (n+1)-itemsets by combining these unique items.
      - Returns a list of all possible (n+1)-itemsets.

      Example:
      If `n_itemset` contains itemsets like [('A', 'B'), ('A', 'C')], the function will generate
      (n+1)-itemsets like [('A', 'B', 'C')] if 'A', 'B', 'C' are the unique items.
  """
  n = len(n_itemset[0])
  return list(combinations(list(set(item for s in n_itemset for item in s)), n + 1))

start_time = time.time()

new_item_set_list = list(set((item,) for transaction in transactions_list for item in transaction))

while new_item_set_list:
  itemset_support = count_item_freq(new_item_set_list)
  freq_itemsets = prune_items(itemset_support)
  freq_itemset_support.update(freq_itemsets)
  if len(freq_itemsets) == 0:
    break
  new_item_set_list = make_n_itemset(list(freq_itemsets.keys()))

for itemset, sup in freq_itemset_support.items():
  print(itemset, sup)

('Microsoft Office',) 0.55
('Speakers',) 0.55
('Flash Drive',) 0.65
('Anti-Virus',) 0.7
('Printer',) 0.5
('Lab Top',) 0.6
('Lab Top Case',) 0.7
('Microsoft Office', 'Flash Drive') 0.55
('Flash Drive', 'Printer') 0.5
('Flash Drive', 'Anti-Virus') 0.5
('Lab Top', 'Lab Top Case') 0.5
('Lab Top', 'Anti-Virus') 0.5
('Lab Top Case', 'Anti-Virus') 0.6


**Mining Association Rules**

In [32]:
index = 1
for itemset, sup in freq_itemset_support.items():
  if len(itemset) < 2:
    continue
  for i in range(1, len(itemset)):
    for antecedent in combinations(itemset, i):
      consequent = tuple(set(itemset) - set(antecedent))
      conf = freq_itemset_support[itemset] / freq_itemset_support[antecedent]
      if conf >= confidence:
        print("Rule ", index, ": ", antecedent, "->", consequent)
        print("Confidence: ", conf*100, "%")
        print("Support: ", freq_itemset_support[itemset]*100, "%")
        print("\n")
        index += 1

end_time = time.time()
print("Time taken: ", end_time - start_time, " seconds")

Rule  1 :  ('Microsoft Office',) -> ('Flash Drive',)
Confidence:  100.0 %
Support:  55.00000000000001 %


Rule  2 :  ('Flash Drive',) -> ('Microsoft Office',)
Confidence:  84.61538461538461 %
Support:  55.00000000000001 %


Rule  3 :  ('Flash Drive',) -> ('Printer',)
Confidence:  76.92307692307692 %
Support:  50.0 %


Rule  4 :  ('Printer',) -> ('Flash Drive',)
Confidence:  100.0 %
Support:  50.0 %


Rule  5 :  ('Flash Drive',) -> ('Anti-Virus',)
Confidence:  76.92307692307692 %
Support:  50.0 %


Rule  6 :  ('Anti-Virus',) -> ('Flash Drive',)
Confidence:  71.42857142857143 %
Support:  50.0 %


Rule  7 :  ('Lab Top',) -> ('Lab Top Case',)
Confidence:  83.33333333333334 %
Support:  50.0 %


Rule  8 :  ('Lab Top Case',) -> ('Lab Top',)
Confidence:  71.42857142857143 %
Support:  50.0 %


Rule  9 :  ('Lab Top',) -> ('Anti-Virus',)
Confidence:  83.33333333333334 %
Support:  50.0 %


Rule  10 :  ('Anti-Virus',) -> ('Lab Top',)
Confidence:  71.42857142857143 %
Support:  50.0 %


Rule  11 :  (